In [4]:
from neo4j import GraphDatabase
import pandas as pd


def read_csv(csv_file):
    data = pd.read_csv(csv_file)
    return data.to_dict(orient='records')

def import_data_to_neo4j(uri, username, password, universities_csv, courses_csv):
    driver = GraphDatabase.driver(uri, auth=(username, password))
    print("Connected")

    universities_data = read_csv(universities_csv)

    university_query = """
    UNWIND $data as row
    MERGE (u:University {id: row.id})
    SET u.name = row.name,
        u.country = row.country
    """

    def create_universities(tx, universities_data):
        tx.run(university_query, data=universities_data)

    courses_data = read_csv(courses_csv)

    course_query = """
    UNWIND $data as row
    MATCH (u:University {id: row.uni})
    MERGE (c:Course {id: row.id})
    SET c.name = row.name,
        c.year = row.year,
        c.semester = row.semester
    MERGE (c)-[:BELONGS_TO]->(u)
    """

    def create_courses(tx, courses_data):
        tx.run(course_query, data=courses_data)

    with driver.session() as session:
        session.write_transaction(create_universities, universities_data)
        session.write_transaction(create_courses, courses_data)

    driver.close()

uri = "bolt://localhost:7687"
username = "neo4j"
password = "12345678"


universities_csv = "univ.csv"
courses_csv = "course.csv"

import_data_to_neo4j(uri, username, password, universities_csv, courses_csv)


Connected


C:\Users\Ghada\AppData\Local\Temp\ipykernel_29188\3580032020.py:49: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_universities, universities_data)
C:\Users\Ghada\AppData\Local\Temp\ipykernel_29188\3580032020.py:50: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_courses, courses_data)
